In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.join(os.path.abspath(''), '../'))

from models.graphsage_model import GraphSAGE
from pprint import pprint
from dataset.data_loader import DataLoader, playtime_forever_users_games_edge_scoring_function

import networkx as nx
from torch_geometric.data import Data
import torch
from torch_geometric.nn import SAGEConv
import pickle
from utils.utils import get_game_name_and_scores

/Users/jackson/.pyenv/versions/3.11.3/envs/ml_project_3.11.3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%reload_ext autoreload

In [3]:
# data_loader = DataLoader(num_users_to_load_per_snowball=100, users_games_edge_scoring_function=playtime_forever_users_games_edge_scoring_function,user_game_edge_embeddings=['playtime_forever'], cache_local_dataset=True, game_embeddings=['name'])
data_loader = DataLoader(users_games_edge_scoring_function=playtime_forever_users_games_edge_scoring_function,user_game_edge_embeddings=['playtime_forever'], cache_local_dataset=True, game_embeddings=['numReviews', 'avgReviewScore', 'price', 'numFollowers', 'genres', 'tags', 'name'])
data_loader.load_random_edge_train_test_split(train_percentage=0.9, test_percentage=0.1, seed=0)


In [8]:
%reload_ext autoreload
save_file_name = 'test_graphsage_model'
model = GraphSAGE(hidden_channels=50, aggr='mean', save_file_name=save_file_name, nn_save_name='best', num_epochs=50, batch_percent=0.1, learning_rate=1e-2, weight_decay=0, seed=12412)
model.set_data_loader(data_loader)
model.train(debug=False)
model.save(save_file_name, overwrite=True)

Total Learnable Parameters: 72801


Training:  10%|█         | 5/50 [02:59<26:46, 35.70s/it]

In [ ]:
display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(76561198835352289)[:10]))
display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(76561198835352289)[-10:]))

,id,name,score
0,789760,DEAD OR SCHOOL,3.709260
1,1490410,The fairy tale you don't know,3.707928
2,1260820,Third Crisis,3.707332
3,930210,HuniePop 2: Double Date,3.706984
4,1217390,Some Some Convenience Store,3.706026
5,1072640,Beat Blast,3.705884
6,2495450,Winter Memories,3.703913
7,407330,Sakura Dungeon,3.703710
8,630550,Faulty Apprentice,3.702140
9,774181,Rhythm Doctor,3.701889


,id,name,score
0,440,Team Fortress 2,3.465204
1,444200,World of Tanks Blitz,3.464998
2,578080,PUBG: BATTLEGROUNDS,3.457726
3,230410,Warframe,3.455014
4,444090,Paladins®,3.454465
5,433850,Z1 Battle Royale,3.454292
6,2357570,Overwatch® 2,3.451211
7,730,Counter-Strike 2,3.444271
8,552990,World of Warships,3.436205
9,236390,War Thunder,3.414328


In [ ]:
model2 = GraphSAGE()
model2.load('test_graphsage_model', load_published_model=False)
model2.set_data_loader(data_loader)
display(get_game_name_and_scores(data_loader, model2.score_and_predict_n_games_for_user(76561198835352289)[:10]))
display(get_game_name_and_scores(data_loader, model2.score_and_predict_n_games_for_user(76561198835352289)[-10:]))

,id,name,score
0,789760,DEAD OR SCHOOL,3.770394
1,1490410,The fairy tale you don't know,3.769062
2,1260820,Third Crisis,3.768466
3,930210,HuniePop 2: Double Date,3.768118
4,1217390,Some Some Convenience Store,3.767160
5,1072640,Beat Blast,3.767018
6,2495450,Winter Memories,3.765047
7,407330,Sakura Dungeon,3.764844
8,630550,Faulty Apprentice,3.763274
9,774181,Rhythm Doctor,3.763023


,id,name,score
0,440,Team Fortress 2,3.439802
1,444200,World of Tanks Blitz,3.439595
2,578080,PUBG: BATTLEGROUNDS,3.432323
3,230410,Warframe,3.429612
4,444090,Paladins®,3.429063
5,433850,Z1 Battle Royale,3.428890
6,2357570,Overwatch® 2,3.425809
7,730,Counter-Strike 2,3.418869
8,552990,World of Warships,3.410803
9,236390,War Thunder,3.388926
